In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
import matplotlib.pyplot as plt
#import plotly.express as px
import glob
from tqdm import tqdm
from sklearn import preprocessing
import sweetviz as sv

In [2]:
# import data
url = '../data/viirs-snpp_2021_Australia.csv'
fileName = '../data/viirs-snpp_2021_Australia'
aus_fires = pd.read_csv(url, parse_dates=['acq_date'])
aus_fires.shape
aus_fires.head()
aus_fires.drop_duplicates()
aus_fires.info()

(812052, 15)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-37.868965,145.241455,330.10,0.47,0.40,2021-01-01,354,N,VIIRS,l,1,303.56,2.36,D,0
1,-35.096336,138.580673,341.55,0.63,0.54,2021-01-01,354,N,VIIRS,n,1,301.13,3.57,D,0
2,-28.734032,148.792267,355.01,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,313.61,24.69,D,0
3,-25.566595,146.441620,356.92,0.39,0.36,2021-01-01,354,N,VIIRS,n,1,314.58,10.19,D,0
4,-26.489180,148.390457,340.80,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,310.51,4.52,D,0


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-37.868965,145.241455,330.10,0.47,0.40,2021-01-01,354,N,VIIRS,l,1,303.56,2.36,D,0
1,-35.096336,138.580673,341.55,0.63,0.54,2021-01-01,354,N,VIIRS,n,1,301.13,3.57,D,0
2,-28.734032,148.792267,355.01,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,313.61,24.69,D,0
3,-25.566595,146.441620,356.92,0.39,0.36,2021-01-01,354,N,VIIRS,n,1,314.58,10.19,D,0
4,-26.489180,148.390457,340.80,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,310.51,4.52,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812047,-20.612724,116.773460,310.05,0.44,0.63,2021-12-31,1806,N,VIIRS,n,1,295.76,2.19,N,2
812048,-21.697954,115.002548,305.72,0.35,0.57,2021-12-31,1806,N,VIIRS,n,1,294.84,0.77,N,2
812049,-20.597576,116.783463,331.56,0.44,0.63,2021-12-31,1806,N,VIIRS,n,1,295.79,2.24,N,2
812050,-20.821804,115.386932,305.26,0.36,0.58,2021-12-31,1806,N,VIIRS,n,1,294.65,0.43,N,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812052 entries, 0 to 812051
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   latitude    812052 non-null  float64       
 1   longitude   812052 non-null  float64       
 2   bright_ti4  812052 non-null  float64       
 3   scan        812052 non-null  float64       
 4   track       812052 non-null  float64       
 5   acq_date    812052 non-null  datetime64[ns]
 6   acq_time    812052 non-null  int64         
 7   satellite   812052 non-null  object        
 8   instrument  812052 non-null  object        
 9   confidence  812052 non-null  object        
 10  version     812052 non-null  int64         
 11  bright_ti5  812052 non-null  float64       
 12  frp         812052 non-null  float64       
 13  daynight    812052 non-null  object        
 14  type        812052 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(3), object(

In [7]:
aus_fires['latitude'] = pd.to_numeric(aus_fires['latitude'], errors='coerce')
aus_fires['longitude'] = pd.to_numeric(aus_fires['longitude'], errors='coerce')
aus_fires['brightness'] = pd.to_numeric(aus_fires['brightness'], errors='coerce')
aus_fires['acq_date'] = pd.to_datetime(aus_fires['acq_date'], errors='coerce')
aus_fires['scan'] = pd.to_numeric(aus_fires['scan'], errors='coerce')
aus_fires['track'] = pd.to_numeric(aus_fires['track'], errors='coerce')
aus_fires['bright_t31'] = pd.to_numeric(aus_fires['bright_t31'], errors='coerce')
aus_fires['frp'] = pd.to_numeric(aus_fires['frp'], errors='coerce')
aus_fires['confidence'] = pd.to_numeric(aus_fires['confidence'], errors='coerce')

In [8]:
aus_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5236764 entries, 0 to 5236763
Data columns (total 15 columns):
 #   Column      Dtype         
---  ------      -----         
 0   latitude    float64       
 1   longitude   float64       
 2   brightness  float64       
 3   scan        float64       
 4   track       float64       
 5   acq_date    datetime64[ns]
 6   acq_time    object        
 7   satellite   object        
 8   instrument  object        
 9   confidence  float64       
 10  version     object        
 11  bright_t31  float64       
 12  frp         float64       
 13  daynight    object        
 14  type        object        
dtypes: datetime64[ns](1), float64(8), object(6)
memory usage: 599.3+ MB


In [9]:
aus_fires.rename({'acq_date': 'time'}, axis=1, inplace=True)
aus_fires['est_fire_area'] = aus_fires['scan'] * aus_fires['track']
aus_fires['est_brightness'] = (aus_fires['brightness'] + aus_fires['bright_t31'])/2
aus_fires.latitude = aus_fires.latitude.round(1)
aus_fires.longitude = aus_fires.longitude.round(1)
aus_fires = aus_fires[aus_fires.type==0]

aus_fires.shape
aus_fires.head()

(3909037, 17)

,latitude,longitude,brightness,scan,track,time,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,est_fire_area,est_brightness
65536,-33.9,115.7,305.0,3.3,1.7,2001-04-21,316,Terra,MODIS,41.0,6.2,294.3,26.8,D,0,5.61,299.65
65537,-42.9,147.9,317.9,1.4,1.2,2001-04-21,1338,Terra,MODIS,96.0,6.2,280.7,29.4,N,0,1.68,299.30
65538,-42.9,147.9,329.0,1.3,1.2,2001-04-21,1338,Terra,MODIS,100.0,6.2,281.8,47.3,N,0,1.56,305.40
65539,-35.3,144.0,303.4,1.2,1.1,2001-04-21,1340,Terra,MODIS,55.0,6.2,281.8,11.7,N,0,1.32,292.60
65540,-32.6,146.4,301.1,1.8,1.3,2001-04-21,1341,Terra,MODIS,38.0,6.2,280.5,19.3,N,0,2.34,290.80


In [10]:
fires = aus_fires[['latitude', 'longitude', 'time','confidence',
                           'est_fire_area','est_brightness','frp']].copy()
## Add fire_count column
count = fires.groupby(['latitude', 'longitude', 'time']).size().reset_index().rename(columns={0:'fire_count'})
fire_copy = fires.merge(count,how='outer', on=['latitude', 'longitude', 'time'])

fire_copy

,latitude,longitude,time,confidence,est_fire_area,est_brightness,frp,fire_count
0,-33.9,115.7,2001-04-21,41.0,5.61,299.65,26.8,1
1,-42.9,147.9,2001-04-21,96.0,1.68,299.30,29.4,2
2,-42.9,147.9,2001-04-21,100.0,1.56,305.40,47.3,2
3,-35.3,144.0,2001-04-21,55.0,1.32,292.60,11.7,1
4,-32.6,146.4,2001-04-21,38.0,2.34,290.80,19.3,1
...,...,...,...,...,...,...,...,...
3909032,-26.7,150.7,2020-12-31,35.0,1.32,295.85,4.7,1
3909033,-28.3,122.2,2020-12-31,29.0,3.45,303.00,38.1,1
3909034,-28.3,122.3,2020-12-31,31.0,3.30,299.50,32.8,3
3909035,-28.3,122.3,2020-12-31,65.0,1.32,306.00,20.6,3


In [11]:
fire_copy = fire_copy.groupby(['latitude', 'longitude', 'time'])[['fire_count','confidence','frp','est_fire_area','est_brightness']].mean().reset_index()
fire_copy

,latitude,longitude,time,fire_count,confidence,frp,est_fire_area,est_brightness
0,-43.6,146.8,2007-03-22,2.0,89.500000,200.450000,2.080000,326.800000
1,-43.6,146.8,2014-04-01,1.0,61.000000,53.100000,2.800000,306.250000
2,-43.5,146.8,2007-03-22,2.0,78.000000,56.700000,2.080000,303.575000
3,-43.5,146.8,2014-02-07,3.0,97.666667,162.533333,4.586667,307.616667
4,-43.5,146.8,2014-02-08,2.0,65.000000,108.200000,7.790000,301.625000
...,...,...,...,...,...,...,...,...
1107515,-9.4,142.7,2020-11-12,1.0,0.000000,19.500000,2.340000,306.550000
1107516,-9.3,142.3,2020-11-10,1.0,0.000000,8.500000,1.100000,308.400000
1107517,-9.2,142.2,2003-10-04,1.0,54.000000,12.000000,1.000000,314.000000
1107518,-9.2,142.2,2005-10-09,1.0,39.000000,10.100000,1.000000,311.000000


In [12]:
## Add location count
count = fire_copy.groupby(['latitude', 'longitude']).size().reset_index().rename(columns={0:'loc_count'})
fire_copy = fire_copy.merge(count,how='outer', on=['latitude', 'longitude'])

fire_copy

,latitude,longitude,time,fire_count,confidence,frp,est_fire_area,est_brightness,loc_count
0,-43.6,146.8,2007-03-22,2.0,89.500000,200.450000,2.080000,326.800000,2
1,-43.6,146.8,2014-04-01,1.0,61.000000,53.100000,2.800000,306.250000,2
2,-43.5,146.8,2007-03-22,2.0,78.000000,56.700000,2.080000,303.575000,5
3,-43.5,146.8,2014-02-07,3.0,97.666667,162.533333,4.586667,307.616667,5
4,-43.5,146.8,2014-02-08,2.0,65.000000,108.200000,7.790000,301.625000,5
...,...,...,...,...,...,...,...,...,...
1107515,-9.4,142.7,2020-11-12,1.0,0.000000,19.500000,2.340000,306.550000,10
1107516,-9.3,142.3,2020-11-10,1.0,0.000000,8.500000,1.100000,308.400000,1
1107517,-9.2,142.2,2003-10-04,1.0,54.000000,12.000000,1.000000,314.000000,3
1107518,-9.2,142.2,2005-10-09,1.0,39.000000,10.100000,1.000000,311.000000,3


In [13]:
fire_copy.est_fire_area = fire_copy.est_fire_area.round(1)
fire_copy.est_brightness = fire_copy.est_brightness.round(1)
fire_copy.confidence = fire_copy.confidence.round().astype(int)
fire_copy.frp = fire_copy.frp.round(1)
fire_copy.fire_count = fire_copy.fire_count.round().astype(int)

fire_copy.head(20)

,latitude,longitude,time,fire_count,confidence,frp,est_fire_area,est_brightness,loc_count
0,-43.6,146.8,2007-03-22,2,90,200.4,2.1,326.8,2
1,-43.6,146.8,2014-04-01,1,61,53.1,2.8,306.2,2
2,-43.5,146.8,2007-03-22,2,78,56.7,2.1,303.6,5
3,-43.5,146.8,2014-02-07,3,98,162.5,4.6,307.6,5
4,-43.5,146.8,2014-02-08,2,65,108.2,7.8,301.6,5
5,-43.5,146.8,2014-02-09,3,79,27.0,1.0,312.3,5
6,-43.5,146.8,2020-03-27,1,0,33.6,1.2,306.1,5
7,-43.5,146.9,2007-03-22,2,80,81.0,2.1,308.8,2
8,-43.5,146.9,2020-03-18,1,100,0.0,1.7,327.8,2
9,-43.4,146.8,2003-04-27,1,23,13.9,1.3,291.2,1


In [14]:
## Normalizing

In [15]:
confidence_norm = preprocessing.normalize([fire_copy['confidence']], norm='max')
frp_norm = preprocessing.normalize([fire_copy['frp']], norm='max')
area_norm = preprocessing.normalize([fire_copy['est_fire_area']], norm='max')
count_norm = preprocessing.normalize([fire_copy['fire_count']], norm='max')

In [16]:
fire_copy['ranking'] = confidence_norm[0]*0.4 + frp_norm[0]*0.2 + area_norm[0]*0.2 + count_norm[0]*0.2

In [17]:
fire_copy = fire_copy.sort_values(by=['loc_count'], ascending=False)
fire_copy.head(30)

,latitude,longitude,time,fire_count,confidence,frp,est_fire_area,est_brightness,loc_count,ranking
1023879,-12.5,141.9,2006-09-20,2,40,13.4,2.5,296.6,566,0.216150
1023872,-12.5,141.9,2006-08-18,3,70,35.3,2.3,306.8,566,0.335000
1023874,-12.5,141.9,2006-08-23,8,57,19.7,1.2,314.1,566,0.267074
1023875,-12.5,141.9,2006-08-24,2,74,22.4,1.7,311.7,566,0.336066
1023876,-12.5,141.9,2006-08-27,4,84,41.0,1.7,319.9,566,0.380469
1023877,-12.5,141.9,2006-09-08,2,78,39.6,1.2,321.5,566,0.342761
1023878,-12.5,141.9,2006-09-18,3,75,18.9,2.1,299.9,566,0.349772
1023880,-12.5,141.9,2006-09-27,1,53,17.2,3.8,296.8,566,0.293879
1023881,-12.5,141.9,2006-09-28,2,68,10.0,1.7,299.1,566,0.311263
1023882,-12.5,141.9,2006-09-30,5,71,27.6,2.0,319.4,566,0.335452


In [20]:
import sweetviz as sv
analyze_report = sv.analyze(fire_copy)
analyze_report.show_html('analyze.html', open_browser=False)


Report analyze.html was generated.


In [18]:
fire_copy.to_csv(f"{fileName}_prepared.csv.gz", index=False, compression='gzip')

In [19]:
fire_copy.to_csv(f"{fileName}_prepared.csv", index=False)